In [1]:
import pandas as pd
import numpy as np
import random as rd

from rdkit import Chem
from rdkit.Chem import AllChem, rdFingerprintGenerator

In [2]:
# # take a subset.
# cell2ind = pd.read_csv('../DrugCell/data/cell2ind.txt', header=None, sep='\t')
# # take a whole things
# gene2ind = pd.read_csv('../DrugCell/data/gene2ind.txt', header=None, sep='\t')
# # take a subset.
# drug2ind = pd.read_csv('../DrugCell/data/drug2ind.txt', header=None, sep='\t')
# # take a subset.
# cell2mutation = pd.read_csv('../DrugCell/data/cell2mutation.txt', header=None, sep=',')
# # take a subset.
# drug2fingerprint = pd.read_csv('../DrugCell/data/drug2fingerprint.txt', header=None, sep=',')
# # take a whole things
# drugcell_ont = pd.read_csv('../DrugCell/data/drugcell_ont.txt', header=None, sep='\t')
# # make train, test and validation
# rcellminer = pd.read_csv('../DrugCell/data/rcellminer_test.txt', header=None, sep='\t')

In [3]:
# indexes = list(rcellminer.index)
# rd.Random(42).shuffle(indexes)

In [4]:
# test = indexes[round(len(indexes)*0.8):]
# tmp = indexes[:round(len(indexes)*0.8)]
# val = tmp[:len(test)]
# train = tmp[len(test):]

In [5]:
# rcellminer.iloc[test].to_csv(
#     '../data/test.txt', sep='\t', 
#     header=None, index=None
# )

In [6]:
# rcellminer.iloc[val].to_csv(
#     '../data/val.txt', sep='\t', 
#     header=None, index=None
# )

In [7]:
# rcellminer.iloc[train].to_csv(
#     '../data/train.txt', sep='\t', 
#     header=None, index=None
# )

In [8]:
# Create PubChem_id list
pubchem_id = pd.read_csv('../DrugCell/data/pubchem_id_by_nsc.csv', index_col=0).dropna()
pubchem_id['PUBCHEM_ID'].astype(int).to_csv('../DrugCell/data/pubchem_id.csv', index=None, header=None)
pubchem_id.shape

(16522, 1)

In [9]:
#  Get SMILES from PubChem ID  https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi
SMILES = pd.read_table('../DrugCell/data/SMILES_from_PubchemID.txt', header=None)
SMILES.index = pubchem_id.index
SMILES = SMILES.drop(0, axis=1)
SMILES.columns = ['SMILES']
SMILES.shape

(16522, 1)

## Drug to Index

In [10]:
SMILES.reset_index(drop=True).to_csv(
    '../DrugCell/data/drug2ind_rcell.txt', 
    header=None,
    sep='\t'
)

## mogan finger printing

In [11]:
mfp = np.array([
 np.array(
  AllChem.GetMorganFingerprintAsBitVect(
    Chem.MolFromSmiles(i), 
    useChirality=True, 
    radius=2, 
    nBits=2048
  )) for i in SMILES['SMILES']
])

[15:08:53] WARNING: not removing hydrogen atom without neighbors
[15:08:54] WARNING: not removing hydrogen atom without neighbors


In [12]:
mfp = pd.DataFrame(mfp)
mfp.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
mfp.to_csv(
    '../DrugCell/data/mfp.txt',
    index=None,
    header=None
)

In [14]:
nci60Act = pd.read_csv('../DrugCell/data/nci60Act.csv', index_col=0)

# need to change the name to match to the table.
nci60Act = nci60Act.rename(
    columns={
        'BR:HS 578T': 'BR:HS 578 T', 
        'BR:T-47D': 'BR:T47D',
        'CNS:SF-295': 'CNS:SF295',
        'CNS:SF-268': 'CNS:SF268',
        'CNS:SF-539': 'CNS:SF539',
        'CNS:SNB-19': 'CNS:SNB19',
        'CO:HT29': 'CO:HT 29',
        'CO:SW-620': 'SW620',
        'LE:HL-60(TB)': 'LE:HL-60',
        'ME:LOX IMVI': 'ME:LOXIMVI',
        'LC:A549/ATCC': 'LC:A549',
        'OV:IGROV1': 'OV:IGROV 1',
        'RE:RXF 393': 'RE:RXF-393',
        'RE:TK-10': 'RE:TK10' 
    }
)

# These are not listed in the table and train data
nci60Act = nci60Act.drop([
    'ME:MDA-N', 
    'OV:NCI/ADR-RES',
    'RE:UO-31'
], axis=1)

In [15]:
nci60Act

,BR:MCF7,BR:MDA-MB-231,BR:HS 578 T,BR:BT-549,BR:T47D,CNS:SF268,CNS:SF295,CNS:SF539,CNS:SNB19,CNS:SNB-75,...,OV:SK-OV-3,PR:PC-3,PR:DU-145,RE:786-0,RE:A498,RE:ACHN,RE:CAKI-1,RE:RXF-393,RE:SN12C,RE:TK10
1,-0.271314,-0.303539,-0.815183,-0.231499,1.934731,-0.357577,-0.719253,-0.380720,-1.281589,-0.175915,...,-0.868240,-0.403983,-0.434885,-0.519867,-1.648059,1.657273,-0.269717,0.002290,-0.390689,-0.379420
17,-0.354110,-0.304675,-0.222024,1.483613,1.509397,0.335572,0.424922,1.140166,-0.941890,0.330808,...,-0.492953,1.302516,-0.941890,0.521200,-0.329639,-0.941890,0.717851,-0.239018,-0.315533,-0.761559
89,NaN,NaN,NaN,NaN,NaN,-0.184194,-1.429903,-0.165433,-1.429903,-0.216723,...,-0.126557,NaN,NaN,0.368433,-0.655707,-0.144006,0.743888,-0.435620,-0.184805,-0.101929
185,NaN,NaN,NaN,NaN,NaN,0.539343,NaN,0.230402,-0.765829,-1.125208,...,-2.151708,NaN,NaN,-0.061612,NaN,NaN,1.675353,0.918722,1.391604,-0.904724
295,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,...,-0.264586,-0.264586,-0.264586,4.822657,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900911,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,...,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,1.594059,-0.167151,-0.167151
900922,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,...,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786
900964,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,...,-0.158754,-0.158754,NaN,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754
900974,-0.132453,-0.132453,-0.132453,NaN,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,...,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,NaN,-0.132453,-0.132453


In [16]:
col = []
for i in nci60Act.columns:
    tmp = i.split(':')
    if len(tmp) == 2:
        col.append(tmp[1])
    else:
        col.append(tmp[0])

nci60Act.columns = col

In [17]:
sanger_cell_lines = pd.read_csv('../DrugCell/data/sanger_to_ccle.csv', index_col=0)
cell2ind = list(pd.read_table('../DrugCell/data/cell2ind.txt', header=None)[1])

In [18]:
ccle_dict = {
    i.upper():j for i,j in zip(
        sanger_cell_lines['Sanger name'], 
        sanger_cell_lines['CCLE Label']
    )
}

In [19]:
nci60Act.columns = [
    ccle_dict[i.replace(' ', '-')] for i in nci60Act.columns
]
nci60Act = nci60Act[list(set(cell2ind) & set(nci60Act.columns))]

In [20]:
nci60Act.head()

,RPMI8226_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SF539_CENTRAL_NERVOUS_SYSTEM,CAKI1_KIDNEY,OVCAR8_OVARY,HCC2998_LARGE_INTESTINE,SKMEL5_SKIN,HCT116_LARGE_INTESTINE,786O_KIDNEY,HOP62_LUNG,A549_LUNG,...,DU145_PROSTATE,EKVX_LUNG,NCIH23_LUNG,MDAMB435S_SKIN,PC3_PROSTATE,MOLT4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SW620_LARGE_INTESTINE,NCIH322_LUNG,BT549_BREAST,M14_SKIN
1,1.504554,-0.380720,-0.269717,-0.413599,-0.490868,-0.108730,0.732038,-0.519867,-0.841210,-0.826223,...,-0.434885,-0.264901,-0.600872,-0.386710,-0.403983,1.540716,1.294131,-0.802747,-0.231499,-0.508668
17,1.698491,1.140166,0.717851,-0.572734,-0.941890,-0.941890,-0.941890,0.521200,-0.941890,-0.941890,...,-0.941890,-0.676210,-0.925414,-0.941890,1.302516,1.686372,-1.029408,-0.786423,1.483613,-0.941890
89,1.191198,-0.165433,0.743888,0.185498,-0.163225,-0.118614,0.347687,0.368433,-1.429903,-0.433001,...,NaN,-0.063351,-0.312738,NaN,NaN,0.549935,0.658260,-1.429903,NaN,0.430283
185,-0.024337,0.230402,1.675353,0.095648,0.476776,1.375379,NaN,-0.061612,0.347152,0.500858,...,NaN,-0.147899,0.170420,NaN,NaN,1.675353,0.683279,0.332463,NaN,-0.533247
295,NaN,-0.264586,-0.264586,3.362653,-0.264586,-0.264586,-0.264586,4.822657,-0.264586,-0.264586,...,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586


In [21]:
base = pd.DataFrame()
for i in nci60Act.columns:
    tmp = pd.DataFrame((nci60Act[i].dropna()))
    tmp.columns = ['Drug Response']
    tmp['Cell Line'] = [i]*len(tmp)
    tmp = pd.merge(tmp, SMILES, left_index=True, right_index=True)
    base = pd.concat([base, tmp])
    
base = base[['Cell Line', 'SMILES', 'Drug Response']]
base = base.reset_index(drop=True)

In [22]:
print(len(base['Cell Line'].unique()))
print(len(base['SMILES'].unique()))

55
16522


In [23]:
pd.DataFrame(base['Cell Line'].value_counts())

,Cell Line
A549_LUNG,16168
OVCAR8_OVARY,16145
U251MG_CENTRAL_NERVOUS_SYSTEM,16087
HCT15_LARGE_INTESTINE,16082
HCT116_LARGE_INTESTINE,16079
SW620_LARGE_INTESTINE,16073
SF295_CENTRAL_NERVOUS_SYSTEM,16024
KM12_LARGE_INTESTINE,16006
COLO205_LARGE_INTESTINE,16003
SF268_CENTRAL_NERVOUS_SYSTEM,15993


In [24]:
indexes = list(base.index)
rd.Random(42).shuffle(indexes)

In [25]:
test = indexes[round(len(indexes)*0.8):]
tmp = indexes[:round(len(indexes)*0.8)]
val = tmp[:len(test)]
train = tmp[len(test):]

In [26]:
train = base.iloc[train]
test = base.iloc[test]
val = base.iloc[val]

In [27]:
print('train: ' + str(train.shape))
print('val: ' + str(val.shape))
print('test: ' + str(test.shape))

train: (498877, 3)
val: (166292, 3)
test: (166292, 3)


In [28]:
df = pd.merge(
    pd.merge(
        pd.DataFrame(train['Cell Line'].value_counts()), 
        pd.DataFrame(val['Cell Line'].value_counts()), 
        left_index=True, right_index=True
    ), pd.DataFrame(test['Cell Line'].value_counts()), 
    left_index=True, right_index=True
)
df.columns = ['train', 'val', 'test']
df.loc['total'] = np.sum(df, axis=0) 

In [29]:
df

,train,val,test
U251MG_CENTRAL_NERVOUS_SYSTEM,9781,3140,3166
A549_LUNG,9739,3116,3313
SW620_LARGE_INTESTINE,9671,3225,3177
KM12_LARGE_INTESTINE,9669,3120,3217
OVCAR8_OVARY,9668,3243,3234
COLO205_LARGE_INTESTINE,9609,3217,3177
HCT15_LARGE_INTESTINE,9607,3202,3273
HCT116_LARGE_INTESTINE,9596,3215,3268
SF295_CENTRAL_NERVOUS_SYSTEM,9587,3227,3210
ACHN_KIDNEY,9583,3118,3136


In [30]:
test.to_csv(
    '../DrugCell/data/test_rcell.txt', sep='\t', 
    header=None, index=None
)

val.to_csv(
    '../DrugCell/data/val_rcell.txt', sep='\t', 
    header=None, index=None
)

train.to_csv(
    '../DrugCell/data/train_rcell.txt', sep='\t', 
    header=None, index=None
)